In [55]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pygcransac
from time import time

correspondences = np.loadtxt('../build/data/rigid_pose_example/rigid_pose_example_points.txt')
gt_pose = np.loadtxt('../build/data/rigid_pose_example/rigid_pose_example_gt.txt')
threshold = 0.03

print("Number of correspondences loaded = ", str(len(correspondences)))

Number of correspondences loaded =  4138


In [56]:
def verify_pygcransac(corrs, threshold, min_iters=1000, max_iters=5000):    
    n = len(corrs)
    
    threshold = 100.0
    pose, mask = pygcransac.findRigidTransform(
        np.ascontiguousarray(corrs), 
        probabilities = [],
        threshold = threshold, 
        neighborhood_size = 4,
        sampler = 1,
        min_iters = min_iters,
        max_iters = max_iters,
        spatial_coherence_weight = 0.0,
        use_space_partitioning = True,
        neighborhood = 0,
        conf = 0.999,
        use_sprt = False)    
    return pose, mask

def tranform_points(corrs, T):
    n = len(corrs)
    points1 = np.float32([corrs[i][0:3] for i in np.arange(n)]).reshape(-1,3)
    points2 = np.float32([corrs[i][3:6] for i in np.arange(n)]).reshape(-1,3)
    
    transformed_corrs = np.zeros((corrs.shape[0], 6))

    for i in range(n):
        p1 = np.append(correspondences[i][:3], 1)
        p2 = p1.dot(T)
        transformed_corrs[i][:3] = p2[:3]
        transformed_corrs[i][3:] = corrs[i][3:]
    return transformed_corrs
    

def calculate_error(gt_pose, est_pose):
    R2R1 = np.dot(gt_pose[:3, :3].T, est_pose[:3, :3])
    cos_angle = max(-1.0, min(1.0, 0.5 * (R2R1.trace() - 1.0)))
    
    err_R = np.arccos(cos_angle) * 180.0 / np.pi
    err_t = np.linalg.norm(gt_pose[:3, 3] - est_pose[:3, 3])
    
    return err_R, err_t


In [57]:

ground_truth_T = gt_pose[:4, :]

# Translating the points so there are no negative coordinates.
# This is only important if the space partitioning technique is used to
# accelerate the robust estimation, or when the spatial coherence term is >0.
min_coordinates = np.min(correspondences, axis=0)
T1 = np.array([[1,0,0,0], [0,1,0,0], [0,0,1,0], [-min_coordinates[0], -min_coordinates[1], -min_coordinates[2], 1]])
T2inv = np.array([[1,0,0,0], [0,1,0,0], [0,0,1,0], [min_coordinates[3], min_coordinates[4], min_coordinates[5], 1]])
transformed_correspondences = correspondences - min_coordinates

t = time()
gc_T, gc_mask = verify_pygcransac(transformed_correspondences, threshold)
if gc_T is None:
    gc_T = np.eye(4)
else:
    gc_T = T1 @ gc_T @ T2inv
    gc_T = gc_T.T
    
print("Run-time = ", time() - t, ' sec')

err_R, err_t = calculate_error(ground_truth_T, gc_T)

print ('Inlier number = ', np.sum(gc_mask))
print ('Rotation error = ', err_R, '°')
print ('Translation error = ', err_t, ' cm')

Run-time =  0.14854717254638672  sec
Inlier number =  4138
Rotation error =  1.8918567448909323 °
Translation error =  0.0861913438662492  cm
